In [16]:
%pip install \
    datasets --quiet

from datasets import load_dataset

Note: you may need to restart the kernel to use updated packages.


In [35]:
datasets = {
    'dialog': load_dataset('air_dialogue'),
    'tweets': load_dataset('tweet_eval', 'emoji'),
    'reviews': load_dataset('amazon_reviews_multi'),
    # wikipedia set is very large - setting to streaming to load only on demand
    'wikipedia': load_dataset("google/wit", streaming=True)
}
    


No config specified, defaulting to: air_dialogue/air_dialogue_data
Found cached dataset air_dialogue (/Users/ralgara/.cache/huggingface/datasets/air_dialogue/air_dialogue_data/1.1.0/38932dc01f4fbf071a129beb7c4afd0982306ab8bc7c994f050467014df5e4a5)
100%|██████████| 2/2 [00:00<00:00, 87.65it/s]
Found cached dataset tweet_eval (/Users/ralgara/.cache/huggingface/datasets/tweet_eval/emoji/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343)
100%|██████████| 3/3 [00:00<00:00, 616.66it/s]
No config specified, defaulting to: amazon_reviews_multi/all_languages
Found cached dataset amazon_reviews_multi (/Users/ralgara/.cache/huggingface/datasets/amazon_reviews_multi/all_languages/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609)
100%|██████████| 3/3 [00:00<00:00, 103.67it/s]


In [36]:
datasets.keys()

dict_keys(['dialog', 'tweets', 'reviews', 'wikipedia'])

In [48]:
datasets['dialog']['train'][0].keys()

dict_keys(['action', 'intent', 'timestamps', 'dialogue', 'expected_action', 'search_info', 'correct_sample'])

In [50]:
for s in datasets['dialog']['train']:
    print(s['dialogue'])
    break

['customer: Hello.', 'agent: Hello. How may I help you?', 'customer: Can you help me to change my recent reservation because my trip dates are got postponed?', 'agent: I will help you with that please share your name to proceed further?', 'customer: Edward hall here.', 'agent: Please wait for a while.', 'customer: Sure, take your own time.', 'agent: There is no active reservation found under your name to amend it.', "customer: That's ok, thank you for checking.", 'agent: Thank you for choosing us.']


In [57]:
participants = [
    {
        "participant_id": 1,
        "type": "AGENT",
        "gender": "FEMALE",
        "is_bot": False
    },
    {
        "participant_id": 2,
        "type": "CLIENT",
        "gender": "MALE",
        "is_bot": False
    }
]

segments = [
    {
        "participant_id": 1,
        "text": "Hello, this is Sara",
        "start": 390,
        "end": 1400
    },
    {
        "participant_id": 1,
        "text": "I am calling to make a purchase",
        "start": 1510,
        "end": 2009
    }
]

template = {
    "project_id": None,
    "document": {
        "natural_id": None,
        "source": "huggingface-den",
        "document_date": None,
        "language": "en",
        "attributes": {
            "airport": "IAD",
            "airline": "AA"
        },
        "verbatims": [
            {
                "call": {
                    "verbatim_types": [
                        "agentverbatim",
                        "clientverbatim"
                    ],
                    "body": {
                        "duration": None,
                        "total_silence": None,
                        "total_dead_air": None,
                        "total_overtalk": None,
                        "total_hesitation": None,
                        "percent_silence": 25.0,
                        "source_system": "source?",
                        "special_events": [
                            {
                                "type": "HOLD",
                                "start": 9020,
                                "end": 13010
                            }
                        ],
                        "participants": participants,
                        "segment_type": "SENTENCE",
                        "segments": segments
                    }
                },
                "all_relations": True,
                "reason_enabled": True
            }
        ]
    }
}

In [83]:
import re

def generate_doc():
    for s in datasets['dialog']['train']:
        doc = template
        segments = []
        ts = s['timestamps']
        ts0 = ts[0]
        for i, turn in enumerate(s['dialogue']):
            participant, sentence = re.compile("(^[a-z]+:) (.*)").match(turn).groups()
            out = {
                "participant_id": (i % 2) + 1,
                "text": turn,
                "start": ts[i] - ts0,
                "end": ts[i] - ts0 + len(turn)
            }
            segments.append(out)
        doc['document']['verbatims'][0]['call']['body']['segments'] = segments
        return doc
    
generate_doc()

{'project_id': None,
 'document': {'natural_id': None,
  'source': 'huggingface-den',
  'document_date': None,
  'language': 'en',
  'attributes': {'carrier': 'Postman', 'caller': 'Mac'},
  'verbatims': [{'call': {'verbatim_types': ['agentverbatim',
      'clientverbatim'],
     'body': {'duration': None,
      'total_silence': None,
      'total_dead_air': None,
      'total_overtalk': None,
      'total_hesitation': None,
      'percent_silence': 25.0,
      'source_system': 'source?',
      'special_events': [{'type': 'HOLD', 'start': 9020, 'end': 13010}],
      'participants': [{'participant_id': 1,
        'type': 'AGENT',
        'gender': 'FEMALE',
        'is_bot': False},
       {'participant_id': 2,
        'type': 'CLIENT',
        'gender': 'MALE',
        'is_bot': False}],
      'segment_type': 'SENTENCE',
      'segments': [{'participant_id': 1,
        'text': 'customer: Hello.',
        'start': 0,
        'end': 16},
       {'participant_id': 2,
        'text': 'agent